In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder

NUM_CLASSES = 2
CLASSES = np.array(['benign', 'malware'])
DATASET_DIR = "dataset/"
VECTOR_LENGTH = 256 * 256

def csvToVector(file_path):
    data = pd.read_csv(file_path)
    data_encoded = LabelEncoder().fit_transform(data['SYSTEM_CALL'])
    vector = np.zeros(VECTOR_LENGTH, dtype=np.uint8)
    syscall_nums = min(len(data_encoded), VECTOR_LENGTH)
    vector[:syscall_nums] = data_encoded[:syscall_nums]

    return vector

def load_data(dataset_dir):
    x = []
    y = []

    classes = [class_name for class_name in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, class_name))]
    for class_idx, class_name in enumerate(classes):
        class_dir = os.path.join(dataset_dir, class_name)
        for file_name in os.listdir(class_dir):
            if file_name.endswith('.csv'):
                file_path = os.path.join(class_dir, file_name)
                vector = csvToVector(file_path)
                x.append(vector)
                y.append(class_idx)
                
    x = np.array(x)
    y = np.array(y)
    return x, y


In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dense, Conv1D, MaxPooling1D
from keras.layers import  BatchNormalization, LeakyReLU, Dropout, Activation

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [3]:
X, y = load_data(DATASET_DIR)

In [4]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

y_train = to_categorical(y_train, NUM_CLASSES)
y_val = to_categorical(y_val, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

In [5]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(220, 65536)
(119, 65536)
(220, 2)
(119, 2)


In [6]:
input_layer = Input(shape=(VECTOR_LENGTH, 1))

x = Conv1D(filters=16, kernel_size=3, padding='same')(input_layer)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = MaxPooling1D(pool_size=2)(x)

x = Conv1D(filters=32, kernel_size=3, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = MaxPooling1D(pool_size=2)(x)

x = Flatten()(x)

x = Dense(128)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate=0.5)(x)

x = Dense(NUM_CLASSES)(x)
output_layer = Activation('softmax')(x)

model = Model(input_layer, output_layer)

opt = Adam(learning_rate=0.0005)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 65536, 1)]        0         
                                                                 
 conv1d (Conv1D)             (None, 65536, 16)         64        
                                                                 
 batch_normalization (BatchN  (None, 65536, 16)        64        
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 65536, 16)         0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 32768, 16)        0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 32768, 32)         1568  

In [10]:
model.fit(X_train, 
          y_train,
          validation_data=(X_val, y_val),
          batch_size=64, 
          epochs=100, 
          shuffle=True
)

Epoch 1/100
4/4 [==============================] - 3s 557ms/step - loss: 0.1530 - accuracy: 0.9545 - val_loss: 0.6905 - val_accuracy: 0.4464
Epoch 2/100
4/4 [==============================] - 4s 1s/step - loss: 0.1529 - accuracy: 0.9591 - val_loss: 0.6909 - val_accuracy: 0.4464
Epoch 3/100
4/4 [==============================] - 4s 1s/step - loss: 0.1692 - accuracy: 0.9636 - val_loss: 0.6911 - val_accuracy: 0.4464
Epoch 4/100
4/4 [==============================] - 5s 1s/step - loss: 0.1936 - accuracy: 0.9636 - val_loss: 0.6885 - val_accuracy: 0.4464
Epoch 5/100
4/4 [==============================] - 5s 1s/step - loss: 0.1607 - accuracy: 0.9636 - val_loss: 0.6863 - val_accuracy: 0.4464
Epoch 6/100
1/4 [======>.......................] - ETA: 3s - loss: 0.0808 - accuracy: 0.9844

KeyboardInterrupt: 